In [1]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import catboost as cb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler  
from sklearn.preprocessing import scale
warnings.filterwarnings('ignore')

C:\Users\elton\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
train_df = pd.read_csv('train2.csv')
test_df = pd.read_csv('test2.csv')

In [3]:
train_df.describe()

,w,y,tariff.plan,age,activ.area,activ.chan,q01.out.ch.peak,q01.out.dur.peak,q01.out.val.peak,q01.out.ch.offpeak,...,q09.out.ch.peak,q09.out.dur.peak,q09.out.val.peak,q09.out.ch.offpeak,q09.out.dur.offpeak,q09.out.val.offpeak,q09.in.ch.tot,q09.in.dur.tot,q09.ch.sms,q09.ch.cc
count,6000.0,6000.000000,6000.000000,6000.000000,6000.00000,6000.00000,6000.000000,6000.000000,6000.000000,6000.000000,...,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,1.0,6.275901,6.856333,31.647458,2.05200,5.39800,64.674333,7104.510000,36.040290,9.152667,...,147.923333,17542.657333,79.046743,3.739333,476.037833,1.545787,113.611667,12027.249833,6.022333,0.649000
std,0.0,1.772212,1.460382,9.458852,0.97303,1.37292,115.886391,14944.334111,69.436016,25.777990,...,160.301328,22821.688164,91.153068,15.710968,2363.876164,8.079536,117.712483,13982.499963,33.321633,2.459093
min,1.0,0.000000,3.000000,16.490000,1.00000,2.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-688.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.0,5.111988,7.000000,23.210000,1.00000,5.00000,0.000000,0.000000,0.000000,0.000000,...,43.000000,3500.750000,20.323325,0.000000,0.000000,0.000000,32.000000,2461.000000,0.000000,0.000000
50%,1.0,6.312641,7.000000,30.980000,2.00000,5.00000,15.000000,913.000000,6.528800,0.000000,...,97.000000,9513.000000,48.562700,0.000000,0.000000,0.000000,79.000000,7522.500000,0.000000,0.000000
75%,1.0,7.422523,8.000000,38.910000,3.00000,5.00000,83.000000,7517.750000,43.476225,6.000000,...,196.000000,22544.750000,105.421900,0.000000,0.000000,0.000000,157.000000,16491.750000,1.000000,0.000000
max,1.0,12.106805,8.000000,52.650000,4.00000,9.00000,1962.000000,273727.000000,1262.523300,441.000000,...,1749.000000,274376.000000,1112.451800,303.000000,49724.000000,335.445000,1494.000000,139646.000000,864.000000,59.000000


## Preprocessing 

In [4]:
train_df.head()

,w,y,tariff.plan,payment.method,gender,age,activ.area,activ.chan,vas1,vas2,...,q09.out.ch.peak,q09.out.dur.peak,q09.out.val.peak,q09.out.ch.offpeak,q09.out.dur.offpeak,q09.out.val.offpeak,q09.in.ch.tot,q09.in.dur.tot,q09.ch.sms,q09.ch.cc
0,1,8.282989,7,bank account,M,35.86,1,5,Y,N,...,520,124208,475.7002,0,0,0.0,116,19288,97,1
1,1,4.744932,8,credit card,F,20.00,1,9,N,N,...,24,4175,23.8686,0,0,0.0,30,4031,0,0
2,1,6.947937,7,credit card,F,27.60,3,7,Y,N,...,326,47268,176.1846,0,0,0.0,131,40350,1,1
3,1,7.107425,7,credit card,B,17.54,1,9,N,N,...,29,8826,32.9887,0,0,0.0,25,5007,0,0
4,1,7.736744,7,bank account,B,48.68,2,5,N,N,...,238,30463,127.9026,0,0,0.0,227,27687,0,1


In [5]:
for i in train_df.columns:
    print("{}:{}\n".format(i, train_df[i].unique()))

w:[1]

y:[8.28298869 4.74493213 6.94793707 ... 8.76655015 8.8539511  8.13768818]

tariff.plan:[7 8 3 6 4]

payment.method:['bank account' 'credit card' 'post account']

gender:['M' 'F' 'B']

age:[35.86 20.   27.6  17.54 48.68 33.63 25.31 16.98 20.49 32.85 41.27 29.4
 43.6  23.93 29.6  17.93 29.96 31.01 40.81 17.77 22.98 39.57 23.5  21.44
 34.19 21.67 28.75 35.77 41.57 22.32 20.98 38.42 19.5  18.65 39.21 28.98
 49.08 34.45 18.78 34.98 34.65 50.03 48.19 46.72 43.11 49.34 20.16 28.95
 25.04 34.52 47.04 17.01 17.63 23.24 39.96 48.16 43.73 45.04 25.08 43.44
 19.8  43.93 21.31 40.88 24.42 36.32 25.44 34.75 32.72 23.01 26.68 20.78
 27.96 37.4  17.7  17.47 32.13 42.78 23.18 41.01 45.44 32.62 42.95 38.19
 27.11 46.95 23.27 25.11 35.34 16.78 23.08 36.39 46.59 26.45 41.67 22.22
 34.88 22.16 39.31 24.81 32.03 38.32 31.44 40.65 30.98 34.62 45.11 50.09
 23.67 40.68 26.95 28.45 38.72 32.45 33.37 31.5  36.09 28.13 24.55 29.11
 26.13 26.62 27.04 33.73 21.18 27.9  30.62 22.49 24.32 21.01 22.26 19.18
 23

In [6]:
for i in range(15):
    print(type(train_df.iloc[5,i]))

<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'numpy.int64'>
<class 'str'>
<class 'str'>
<class 'numpy.float64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'str'>
<class 'str'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'numpy.int64'>
<class 'numpy.int64'>


In [7]:
train_df.iloc[:, [2, 6, 7]] = train_df.iloc[:, [2, 6, 7]].astype(str)

In [8]:
train_df.head()

,w,y,tariff.plan,payment.method,gender,age,activ.area,activ.chan,vas1,vas2,...,q09.out.ch.peak,q09.out.dur.peak,q09.out.val.peak,q09.out.ch.offpeak,q09.out.dur.offpeak,q09.out.val.offpeak,q09.in.ch.tot,q09.in.dur.tot,q09.ch.sms,q09.ch.cc
0,1,8.282989,7,bank account,M,35.86,1,5,Y,N,...,520,124208,475.7002,0,0,0.0,116,19288,97,1
1,1,4.744932,8,credit card,F,20.00,1,9,N,N,...,24,4175,23.8686,0,0,0.0,30,4031,0,0
2,1,6.947937,7,credit card,F,27.60,3,7,Y,N,...,326,47268,176.1846,0,0,0.0,131,40350,1,1
3,1,7.107425,7,credit card,B,17.54,1,9,N,N,...,29,8826,32.9887,0,0,0.0,25,5007,0,0
4,1,7.736744,7,bank account,B,48.68,2,5,N,N,...,238,30463,127.9026,0,0,0.0,227,27687,0,1


In [9]:
for i in train_df.columns:
    print("{}:{}\n".format(i, train_df[i].unique()))

w:[1]

y:[8.28298869 4.74493213 6.94793707 ... 8.76655015 8.8539511  8.13768818]

tariff.plan:['7' '8' '3' '6' '4']

payment.method:['bank account' 'credit card' 'post account']

gender:['M' 'F' 'B']

age:[35.86 20.   27.6  17.54 48.68 33.63 25.31 16.98 20.49 32.85 41.27 29.4
 43.6  23.93 29.6  17.93 29.96 31.01 40.81 17.77 22.98 39.57 23.5  21.44
 34.19 21.67 28.75 35.77 41.57 22.32 20.98 38.42 19.5  18.65 39.21 28.98
 49.08 34.45 18.78 34.98 34.65 50.03 48.19 46.72 43.11 49.34 20.16 28.95
 25.04 34.52 47.04 17.01 17.63 23.24 39.96 48.16 43.73 45.04 25.08 43.44
 19.8  43.93 21.31 40.88 24.42 36.32 25.44 34.75 32.72 23.01 26.68 20.78
 27.96 37.4  17.7  17.47 32.13 42.78 23.18 41.01 45.44 32.62 42.95 38.19
 27.11 46.95 23.27 25.11 35.34 16.78 23.08 36.39 46.59 26.45 41.67 22.22
 34.88 22.16 39.31 24.81 32.03 38.32 31.44 40.65 30.98 34.62 45.11 50.09
 23.67 40.68 26.95 28.45 38.72 32.45 33.37 31.5  36.09 28.13 24.55 29.11
 26.13 26.62 27.04 33.73 21.18 27.9  30.62 22.49 24.32 21.01 22.26

In [10]:
for i in range(15):
    print(type(train_df.iloc[5,i]))

<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'numpy.float64'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'numpy.int64'>
<class 'numpy.int64'>


In [11]:
train_df = pd.get_dummies(train_df)

In [12]:
train_df.head()

,w,y,age,q01.out.ch.peak,q01.out.dur.peak,q01.out.val.peak,q01.out.ch.offpeak,q01.out.dur.offpeak,q01.out.val.offpeak,q01.in.ch.tot,...,activ.chan_4,activ.chan_5,activ.chan_6,activ.chan_7,activ.chan_8,activ.chan_9,vas1_N,vas1_Y,vas2_N,vas2_Y
0,1,8.282989,35.86,152,22465,172.9098,241,46404,146.5705,17,...,0,1,0,0,0,0,0,1,1,0
1,1,4.744932,20.00,0,0,0.0000,0,0,0.0000,0,...,0,0,0,0,0,1,1,0,1,0
2,1,6.947937,27.60,125,12538,86.9956,0,0,0.0000,57,...,0,0,0,1,0,0,0,1,1,0
3,1,7.107425,17.54,0,0,0.0000,0,0,0.0000,0,...,0,0,0,0,0,1,1,0,1,0
4,1,7.736744,48.68,280,39788,166.6803,0,0,0.0000,178,...,0,1,0,0,0,0,1,0,1,0


In [13]:
test_df.head()

,tariff.plan,payment.method,gender,age,activ.area,activ.chan,vas1,vas2,q01.out.ch.peak,q01.out.dur.peak,...,q09.out.ch.peak,q09.out.dur.peak,q09.out.val.peak,q09.out.ch.offpeak,q09.out.dur.offpeak,q09.out.val.offpeak,q09.in.ch.tot,q09.in.dur.tot,q09.ch.sms,q09.ch.cc
0,8,bank account,B,25.77,3,5,N,N,74,6628,...,96,9834,60.2974,0,0,0.0000,37,2911,0,0
1,8,post account,M,43.14,1,5,Y,N,36,3229,...,59,5390,52.4023,67,7107,20.8545,215,40155,2,0
2,8,credit card,F,23.67,1,9,N,N,0,0,...,14,1225,7.7215,0,0,0.0000,23,1316,0,2
3,8,credit card,B,34.62,1,5,N,N,48,5643,...,58,4452,28.8002,0,0,0.0000,79,4751,0,0
4,8,bank account,M,22.49,3,5,N,N,0,0,...,106,5337,38.2891,0,0,0.0000,285,18762,0,0


In [14]:
for i in test_df.columns:
    print("{}:{}\n".format(i, test_df[i].unique()))

tariff.plan:[8 7 3 4 6]

payment.method:['bank account' 'post account' 'credit card']

gender:['B' 'M' 'F']

age:[25.77 43.14 23.67 34.62 22.49 22.55 35.7  18.65 24.75 49.57 18.   29.6
 19.63 21.9  32.13 43.44 20.49 44.09 23.01 44.72 38.09 36.06 20.91 34.75
 44.42 45.47 49.83 27.96 48.91 45.34 29.01 20.78 26.68 48.16 19.5  49.4
 24.59 38.78 38.91 43.73 22.78 19.54 38.42 34.42 33.77 49.44 39.57 42.72
 19.01 38.88 21.14 17.44 23.93 37.08 44.95 49.7  42.78 42.95 22.03 43.11
 26.03 38.36 20.72 19.8  33.18 37.27 41.24 47.73 21.83 37.31 28.19 25.6
 21.18 17.77 19.57 47.5  41.57 30.98 35.63 47.63 33.14 21.01 42.42 41.4
 34.45 48.68 23.18 45.18 24.42 17.01 25.34 34.55 26.45 32.59 30.16 19.04
 33.47 27.73 46.95 20.95 26.75 27.63 46.91 26.36 21.67 36.45 29.4  42.09
 16.98 21.77 40.95 31.96 21.44 35.54 27.5  40.16 43.6  27.11 45.24 22.91
 34.88 42.91 26.95 44.59 35.14 41.86 32.55 28.75 35.77 24.62 17.63 40.59
 38.52 25.8  40.   32.95 49.73 33.63 33.5  20.   18.16 18.49 33.37 45.5
 45.44 50.13 25.

In [15]:
for i in range(15):
    print(type(test_df.iloc[5,i]))

<class 'numpy.int64'>
<class 'str'>
<class 'str'>
<class 'numpy.float64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'str'>
<class 'str'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'numpy.int64'>
<class 'numpy.int64'>
<class 'numpy.float64'>
<class 'numpy.int64'>


In [16]:
test_df.iloc[:, [0, 4, 5]] = test_df.iloc[:, [0, 4, 5]].astype(str)

In [17]:
test_df = pd.get_dummies(test_df)

In [18]:
test_df.head()

,age,q01.out.ch.peak,q01.out.dur.peak,q01.out.val.peak,q01.out.ch.offpeak,q01.out.dur.offpeak,q01.out.val.offpeak,q01.in.ch.tot,q01.in.dur.tot,q01.ch.sms,...,activ.chan_4,activ.chan_5,activ.chan_6,activ.chan_7,activ.chan_8,activ.chan_9,vas1_N,vas1_Y,vas2_N,vas2_Y
0,25.77,74,6628,41.5985,0,0,0.0000,14,2385,1,...,0,1,0,0,0,0,1,0,1,0
1,43.14,36,3229,25.6870,85,5156,20.4011,147,13008,4,...,0,1,0,0,0,0,0,1,1,0
2,23.67,0,0,0.0000,0,0,0.0000,0,0,0,...,0,0,0,0,0,1,1,0,1,0
3,34.62,48,5643,32.4113,42,3019,12.0107,67,3196,0,...,0,1,0,0,0,0,1,0,1,0
4,22.49,0,0,0.0000,0,0,0.0000,0,0,0,...,0,1,0,0,0,0,1,0,1,0


## Random forest 

In [19]:
features = [c for c in train_df.columns if c not in ['w', 'y']]
target = train_df['y']
folds = KFold(n_splits=10, shuffle=False, random_state=2319)
scoring = 'mean_squared_error'

In [20]:
rf = RandomForestRegressor(n_estimators=1000, n_jobs=8)

In [21]:
scores = cross_val_score(rf, train_df[features], target, scoring=scoring, cv=folds)

In [22]:
scores.mean()

-1.2584614605014932

## LGBM 

In [23]:
random_state = 42
np.random.seed(random_state)

In [24]:
features = [c for c in train_df.columns if c not in ['w', 'y']]
target = train_df['y']
param = {
    'bagging_freq': 5,               'bagging_fraction': 1,       
    'boost_from_average':'false',    'boost': 'gbdt',
    'feature_fraction': 1,           'learning_rate': 0.01,           'max_depth': -1,                #'min_data_in_leaf': 2,
    'min_sum_hessian_in_leaf': 0.01, 'num_leaves': 15,                'num_threads': 8,               'tree_learner': 'serial',
    'objective': 'regression',       'verbosity': 1,                  #'lambda_l2': 10,                #'max_bin': 511
}

In [25]:
num_round = 20000
folds = KFold(n_splits=10, shuffle=False, random_state=2319)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_df.values, target.values)):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][features], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][features], label=target.iloc[val_idx])
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 100)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][features], num_iteration=clf.best_iteration)
    predictions += clf.predict(test_df[features], num_iteration=clf.best_iteration) / folds.n_splits
print("CV score: {:<8.5f}".format(mean_squared_error(target, oof)))

Fold 0
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[511]	training's l2: 1.01062	valid_1's l2: 1.17916
Fold 1
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[664]	training's l2: 0.959427	valid_1's l2: 1.23726
Fold 2
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[626]	training's l2: 0.970305	valid_1's l2: 1.22474
Fold 3
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[503]	training's l2: 0.991754	valid_1's l2: 1.36831
Fold 4
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[821]	training's l2: 0.902131	valid_1's l2: 1.29589
Fold 5
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[610]	training's l2: 0.971788	valid_1's l2: 1.21132
Fold 6
Training until validation scores don't improve for 1

## Submission 

In [144]:
with open('competition2_lgbm3.txt', 'w') as file_1:
    for number in predictions:
        file_1.write('{0}\n'.format(number))